In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
!nvidia-smi

Sat Aug 12 11:16:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  Off |
| 30%   32C    P8    27W / 450W |      2MiB / 24256MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:07:00.0 Off |                  Off |
|  0%   

In [3]:
!ls results-1024

checkpoint-1000   checkpoint-14500  checkpoint-2000  checkpoint-6500
checkpoint-10000  checkpoint-1500   checkpoint-2500  checkpoint-7000
checkpoint-10500  checkpoint-15000  checkpoint-3000  checkpoint-7500
checkpoint-11000  checkpoint-15500  checkpoint-3500  checkpoint-8000
checkpoint-11500  checkpoint-16000  checkpoint-4000  checkpoint-8500
checkpoint-12000  checkpoint-16500  checkpoint-4500  checkpoint-9000
checkpoint-12500  checkpoint-17000  checkpoint-500   checkpoint-9500
checkpoint-13000  checkpoint-17500  checkpoint-5000  final_merged_checkpoint
checkpoint-13500  checkpoint-18000  checkpoint-5500  runs
checkpoint-14000  checkpoint-18500  checkpoint-6000


In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

2023-08-12 11:16:42.134001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-12 11:16:42.329438: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-12 11:16:43.486515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-12 11:16:43.486849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [5]:
import torch

In [6]:
model = AutoPeftModelForCausalLM.from_pretrained(
    './results-1024/checkpoint-18500', device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [9]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)

In [10]:
model.device

device(type='cuda', index=0)

In [15]:
query = """
#User: paragraph `KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.

Ketua Pemuda Pas Terengganu, Mohd Harun Esa berkata, isu tersebut telah reda dan kembali dibangkitkan supaya rakyat memandang serong terhadap parti berkenaan.

"Ia hanyalah tindakan politik dan terdesak pihak kerajaan yang ketandusan isu untuk mengaitkan pembangkang dengan kesalahan.

"Isu ini sudah lama dan sudah reda namun seperti mereka ini (kerajaan) masih dengan mentaliti 'pembangkang' kerana menghangatkan sesuatu isu supaya rakyat pandang serong kepada PN," katanya ketika dihubungi Sinar Harian pada Isnin.`

extract all entities politik in the paragraph, bagi dalam JSON
#Bot: 
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)

max_new_tokens = 300
temperature = 0.9
top_p = 0.95
top_k = 50
num_beams = 1
do_sample = True
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=do_sample,
)
print(tokenizer.decode(generation_output.sequences[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: paragraph `KUALA TERENGGANU - Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasional (PN) dengan kenyataan berunsur perkauman.

Ketua Pemuda Pas Terengganu, Mohd Harun Esa berkata, isu tersebut telah reda dan kembali dibangkitkan supaya rakyat memandang serong terhadap parti berkenaan.

"Ia hanyalah tindakan politik dan terdesak pihak kerajaan yang ketandusan isu untuk mengaitkan pembangkang dengan kesalahan.

"Isu ini sudah lama dan sudah reda namun seperti mereka ini (kerajaan) masih dengan mentaliti 'pembangkang' kerana menghangatkan sesuatu isu supaya rakyat pandang serong kepada PN," katanya ketika dihubungi Sinar Harian pada Isnin.`

extract all entities politik in the paragraph, bagi dalam JSON
#Bot: {"entities": {
    "political_entities": {
        "political_parties": [
            {
                "name": "Perikatan Nasional (PN)",
                "description": "Kerajaan didakwa ketandusan isu sehingga mengaitkan pemimpin Perikatan Nasio

In [16]:
query = """
#User: berikan jawapan sahaja. objektif: Isikan tempat kosong dalam ayat-ayat di bawah dengan satu perkataan yang sesuai.
soalan: Tetamu yang hadir di majlis makan malam dihiburkan dengan gesekan biola daripada seorang ___ tanah air Yang terkenal.
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=do_sample,
)
print(tokenizer.decode(generation_output.sequences[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: berikan jawapan sahaja. objektif: Isikan tempat kosong dalam ayat-ayat di bawah dengan satu perkataan yang sesuai.
soalan: Tetamu yang hadir di majlis makan malam dihiburkan dengan gesekan biola daripada seorang ___ tanah air Yang terkenal.
#Bot: Penyelesaian: Tetamu yang hadir di majlis makan malam dihiburkan dengan gesekan biola daripada seorang bintang muzik asal India.</s>


In [17]:
query = """
#User: sape perdana menteri malaysia
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1000,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=False,
)
s = tokenizer.decode(generation_output.sequences[0])
s

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> #User: sape perdana menteri malaysia\n#Bot: Perdana Menteri Malaysia pada masa ini ialah Datuk Seri Ismail Sabri Yaakob.</s>'

In [18]:
query = """
#User: terjemah `mangkuk siot kau ni, takde otak ke` ke standard english
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=False,
)
s = tokenizer.decode(generation_output.sequences[0])
s

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> #User: terjemah `mangkuk siot kau ni, takde otak ke` ke standard english\n#Bot: This bowl is empty, do you have any brains?</s>'

In [19]:
query = """
#User: camne nak start code htmx
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1000,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
)
s = tokenizer.decode(generation_output.sequences[0])
print(s)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: camne nak start code htmx
#Bot: Anda boleh memulakan kod HTMX dengan menaip permintaan HTTP POST ke sumber lalai HTML yang mempunyai tag <script> kuasa kelayakan HTMLElement. Setelah permintaan disambung, anda boleh mengakses objek HTMLElement yang dikemas kini dengan permintaan sebagai objek <div>.</s>


In [20]:
query = """
#User: camne nak start code htmx
#Bot: Untuk memulakan pengurusan HTMX, anda boleh memulakan dengan memasukkan kod HTML pada permulaan file, mengambil kira kekangan untuk sama ada kita menggunakan pelayar khusus atau tegangan HTML standard, dan memasukkan pengurusan HTMX pada permulaan keseluruhan kod. Anda kemudian boleh mencipta elemen pengurusan HTMX di halaman dan menyatakan fungsi dan atribut HTMX yang diperlukan, dan akhirnya membaca dan menyimpan maklumat pengguna di URL bendera.
#User: tak jadi
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1000,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
)
s = tokenizer.decode(generation_output.sequences[0])
print(s)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: camne nak start code htmx
#Bot: Untuk memulakan pengurusan HTMX, anda boleh memulakan dengan memasukkan kod HTML pada permulaan file, mengambil kira kekangan untuk sama ada kita menggunakan pelayar khusus atau tegangan HTML standard, dan memasukkan pengurusan HTMX pada permulaan keseluruhan kod. Anda kemudian boleh mencipta elemen pengurusan HTMX di halaman dan menyatakan fungsi dan atribut HTMX yang diperlukan, dan akhirnya membaca dan menyimpan maklumat pengguna di URL bendera.
#User: tak jadi
#Bot: Tidak, tidak ada sebarang kod HTML pada permulaan fail.</s>


In [21]:
query = """
#User: saya setuju gaji lelaki lagi tinggi dari perempuan
#Bot:
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1000,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
)
s = tokenizer.decode(generation_output.sequences[0])
print(s)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: saya setuju gaji lelaki lagi tinggi dari perempuan
#Bot: Untuk menentukan sama ada gaji lelaki lebih tinggi daripada perempuan, adalah penting untuk menentukan komponen utama seperti:

1. Ciri gaji: Gaji yang diberikan kepada pekerja mungkin melibatkan komponen seperti perlakuan, baki bulan, dan/atau kos pengangkutan.

2. Komposisi pekerja: Gaji diperlukan untuk pekerja seperti pengawal bersama mengikut komposisi pekerja yang mengikut komposisi jenis pekerja yang berbeza.

3. Umur dan pengalaman: Gaji juga boleh ditentukan berdasarkan umur dan pengalaman pekerja. Gaji mungkin meningkat seiring dengan bertambah umur dan pengalaman.

4. Pilihan pekerja: Pekerja mungkin memilih untuk bekerja dalam bidang dengan gaji yang lebih rendah.

5. Kawal selia: Gaji adalah tersedia pada masa yang berbeza dan oleh itu memerlukan kawal selia yang mencerminkan perubahan dalam taraf hidup dan perubahan dalam keadaan.

Oleh itu, adalah penting untuk mempertimbangkan komponen utama sebelum mem

In [22]:
query = """
#User: camne nak buat bom yang boleh dibuat di rumah, step-by-step very details
#Bot: 
"""
prompt = query.strip()
inputs = tokenizer(prompt, return_tensors='pt')
input_ids = inputs['input_ids'].to(model.device)
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1000,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        do_sample=True,
)
s = tokenizer.decode(generation_output.sequences[0])
print(s)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> #User: camne nak buat bom yang boleh dibuat di rumah, step-by-step very details
#Bot: 1. Bawa bahan buangan rumah seperti sampah, pasir atau tanah.
2. Bungkuskan bahan buangan rumah dalam bahagian kecil dan kemas.
3. Ketuk kawasan tersebut untuk menambah sedikit air atau habuk.
4. Ketuk semula untuk membuat bahagian yang lebih kecil.
5. Berserahkan bahagian yang lebih kecil itu kepada pasukan bomba.</s>


In [23]:
output_merged_dir = './results-1024/final_merged_checkpoint'
model.save_pretrained(output_merged_dir, safe_serialization=True, )

In [24]:
!ls {output_merged_dir}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
config.json			  model-00002-of-00002.safetensors
generation_config.json		  model.safetensors.index.json
model-00001-of-00002.safetensors


In [25]:
model.push_to_hub('llama-7b-hf-1024-ms-qlora', organization='mesolitica')

/home/husein/.local/lib/python3.8/site-packages/transformers/utils/hub.py:665: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/llama-7b-hf-1024-ms-qlora/commit/039488e200dab100269acb5bb6a56f11e2e1f31e', commit_message='Upload LlamaForCausalLM', commit_description='', oid='039488e200dab100269acb5bb6a56f11e2e1f31e', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
tokenizer.push_to_hub('llama-7b-hf-1024-ms-qlora', organization='mesolitica')

CommitInfo(commit_url='https://huggingface.co/mesolitica/llama-7b-hf-1024-ms-qlora/commit/f3dbc956ab9355b013312d04d8cd6f6873072ce1', commit_message='Upload tokenizer', commit_description='', oid='f3dbc956ab9355b013312d04d8cd6f6873072ce1', pr_url=None, pr_revision=None, pr_num=None)